In [1]:
import requests
import json
from pprint import pprint
from fake_useragent import UserAgent
import  pandas as pd

ua = UserAgent().chrome

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\python36\envs\machine_learning\lib\urllib\request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "C:\python36\envs\machine_learning\lib\http\client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\python36\envs\machine_learning\lib\http\client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\python36\envs\machine_learning\lib\http\client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\python36\envs\machine_learning\lib\http\client.py", line 1026, in _send_output
    self.send(msg)
  File "C:\python36\envs\machine_learning\lib\http\client.py", line 964, in send
    self.connect()
  File "C:\python36\envs\machine_learning

# Request

In [4]:
data = [[524], 270881,1]
r = requests.post('https://www.codingame.com/services/Puzzle/findProgressByIds', data = str(data))
r.text

'[{"id":524,"level":"optim","rank":0,"title":"CGFunge Prime","validatorScore":0,"achievementCount":0,"doneAchievementCount":0,"forumLink":"community-puzzle-cgfunge-prime/116994","total":16,"globalTotal":16,"contributor":{"userId":1500515,"pseudo":"eulerscheZahl","publicHandle":"8374201b6f1d19eb99d61c80351465b65150051","enable":false,"avatar":29379905825543,"cover":26120069129738},"solvedCount":11,"attemptCount":37,"feedback":{"feedbackId":1755,"feedbacks":[0,0,0,1,6]},"optimCriteriaId":"Points","topics":[{"handle":"uncategorized","value":"Sans catégorie","children":[{"handle":"cg-funge","value":"CG Funge","children":[]}]}],"creationTime":1564387127448,"type":"SOLO","mode":"OPTIM","prettyId":"cgfunge-prime","detailsPageUrl":"/multiplayer/optimization/cgfunge-prime","puzzleLeaderboardId":"cgfunge-prime","communityCreation":true}]'

In [5]:
full_data = [list(range(1, 530)), 270881,1]
r = requests.post('https://www.codingame.com/services/Puzzle/findProgressByIds',  data = str(full_data))

In [6]:
pd.DataFrame(json.loads(r.text)).to_csv("results_raw.csv", index=False)

# Load & Cleanup

In [7]:
import pandas as pd
import json
import ast

def compute_total(x):
    json_data = ast.literal_eval(x)
    return sum(json_data["feedbacks"])

def compute_average(x):
    json_data = ast.literal_eval(x)
    total = 0
    for pts, vote in zip([1, 2, 3, 4, 5], json_data["feedbacks"]):
        total += pts * vote
    return total / sum(json_data["feedbacks"])   

In [8]:
df = pd.read_csv("results_raw.csv", encoding = "ISO-8859-1")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 40 columns):
achievementCount        404 non-null int64
attemptCount            404 non-null int64
chatRoom                61 non-null object
codinGamer              245 non-null object
communityCreation       404 non-null bool
contributor             317 non-null object
coverBinaryId           126 non-null float64
creationTime            404 non-null int64
description             68 non-null object
detailsPageUrl          404 non-null object
doneAchievementCount    404 non-null int64
feedback                404 non-null object
forumLink               171 non-null object
globalTotal             44 non-null float64
id                      404 non-null int64
lastActivity            238 non-null float64
league                  11 non-null object
level                   404 non-null object
logoBinaryId            87 non-null float64
mode                    12 non-null object
openChallenge           

In [10]:
# df.testSessionHandle

In [11]:
df['creationTime'] = pd.to_datetime(df['creationTime']*1000000)
df['lastActivity'] = pd.to_datetime(df['lastActivity']*1000000)
       
df["feedbackAvg"] = df["feedback"].apply(compute_average)
df["feedbackCount"] = df["feedback"].apply(compute_total)

df.drop(["chatRoom", "codinGamer", "contributor", 
         "coverBinaryId", "forumLink", "league", 
         "logoBinaryId", "openChallenge", "optimCriteria", 
         "optimCriteriaId", "optimPoints", "prettyId", 
         "previewBinaryId", "puzzleLeaderboardId", "rank", 
         "replayIds", "score", "thumbnailBinaryId",
         "topics", "feedback"], axis=1, inplace=True)  # "testSessionHandle"

df = df.set_index("id")
df = df.sort_index()

In [12]:
df.to_csv("results.csv")

# Analysis

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv("results.csv", encoding = "ISO-8859-1", index_col=0)

In [15]:
df["base"] = "https://www.codingame.com"
df["url"] = df["base"].str.cat(df["detailsPageUrl"])

In [16]:
df.head()

,achievementCount,attemptCount,communityCreation,creationTime,description,detailsPageUrl,doneAchievementCount,globalTotal,lastActivity,level,...,testSessionHandle,title,total,type,validatorScore,xpPoints,feedbackAvg,feedbackCount,base,url
id,,,,,,,,,,,,,,,,,,,,,
1,3,55340,False,2014-07-16 13:25:18.000000000,Le virus de la résistance se propage à travers...,/training/medium/skynet-revolution-episode-1,3,NaN,2019-07-26 15:34:22.688000000,medium,...,37819577b3990cd63492da5e8688b35cfa3b7f5,Skynet Revolution - Episode 1,NaN,SOLO,100,150.0,4.731651,3052,https://www.codingame.com,https://www.codingame.com/training/medium/skyn...
4,2,372399,False,2014-07-23 08:13:21.000000000,"Le marteau de Thor, <i>Mjöllnir</i> a perdu to...",/training/easy/power-of-thor-episode-1,2,NaN,2019-07-27 20:47:17.038000128,easy,...,37542667842fae110bd7bc244b4eebc7927c25c,Power of Thor - Episode 1,NaN,SOLO,100,50.0,4.438970,17942,https://www.codingame.com,https://www.codingame.com/training/easy/power-...
5,2,192017,False,2014-07-23 08:14:24.000000000,Il fait froid ! Saurez-vous trouver la tempéra...,/training/easy/temperatures,2,NaN,2017-07-15 08:56:55.223000064,easy,...,3760408937c17934059ee020ae0f137f92a4ba9,Températures,NaN,CODE,100,50.0,4.236030,13028,https://www.codingame.com,https://www.codingame.com/training/easy/temper...
6,2,110411,False,2014-07-23 08:16:54.000000000,Vous-vous sentez un âme d'artiste ? Mais vous ...,/training/easy/ascii-art,2,NaN,2017-09-17 09:41:33.336999936,easy,...,37612433492b02b8dfb1d5ce584f10b1aa174dd,ASCII Art,NaN,CODE,100,50.0,4.431759,7349,https://www.codingame.com,https://www.codingame.com/training/easy/ascii-art
7,2,86965,False,2014-07-23 08:19:26.000000000,La binaire c'est bien ! Mais le unaire c'est m...,/training/easy/chuck-norris,2,NaN,2018-08-13 15:54:40.916999936,easy,...,3761504deb84534c75a88ff4325125ce6109a7f,Chuck Norris,NaN,CODE,100,50.0,4.261623,5485,https://www.codingame.com,https://www.codingame.com/training/easy/chuck-...


In [17]:
# df[df.achievementCount > df.doneAchievementCount]

In [18]:
# df[df.total<df.globalTotal]

In [19]:
df["percentDone"] = df.solvedCount / df.attemptCount

In [20]:
sub = df[df.communityCreation]
sub = sub[sub.validatorScore < 100]
sub.sort_values(['percentDone'], ascending=[0])

,achievementCount,attemptCount,communityCreation,creationTime,description,detailsPageUrl,doneAchievementCount,globalTotal,lastActivity,level,...,title,total,type,validatorScore,xpPoints,feedbackAvg,feedbackCount,base,url,percentDone
id,,,,,,,,,,,,,,,,,,,,,
417,0,70,True,2018-08-24 12:38:04.794000000,NaN,/training/hard/dumbbells-solver,0,NaN,NaN,hard,...,Dumbbells solver,NaN,CODE,0,50.0,4.545455,11,https://www.codingame.com,https://www.codingame.com/training/hard/dumbbe...,0.928571
421,0,163,True,2018-09-17 17:44:00.363000000,NaN,/training/medium/kaprekars-routine,0,NaN,NaN,medium,...,Kaprekar's routine,NaN,CODE,0,50.0,4.333333,39,https://www.codingame.com,https://www.codingame.com/training/medium/kapr...,0.883436
456,0,88,True,2019-01-18 07:25:21.949000000,NaN,/training/medium/green-valleys,0,NaN,NaN,medium,...,Green Valleys,NaN,CODE,0,50.0,4.318182,22,https://www.codingame.com,https://www.codingame.com/training/medium/gree...,0.875000
173,0,493,True,2016-07-13 12:38:10.257000000,NaN,/training/medium/queneau-numbers,0,NaN,NaN,medium,...,Queneau Numbers,NaN,CODE,0,50.0,4.137931,58,https://www.codingame.com,https://www.codingame.com/training/medium/quen...,0.874239
394,0,171,True,2018-06-17 20:57:24.862000000,NaN,/training/medium/simplified-monopoly-turns-pre...,0,NaN,NaN,medium,...,Simplified Monopoly? Turns Prediction,NaN,CODE,0,50.0,4.424242,33,https://www.codingame.com,https://www.codingame.com/training/medium/simp...,0.865497
400,0,194,True,2018-06-26 11:26:41.371000000,NaN,/training/medium/bulgarian-solitaire,0,NaN,NaN,medium,...,Bulgarian solitaire,NaN,CODE,0,50.0,4.321429,28,https://www.codingame.com,https://www.codingame.com/training/medium/bulg...,0.860825
434,0,85,True,2018-10-15 18:20:23.048000000,NaN,/training/medium/enigma---3-rotors-without-plu...,0,NaN,NaN,medium,...,Enigma - 3 rotors without plugboard,NaN,CODE,0,50.0,3.857143,21,https://www.codingame.com,https://www.codingame.com/training/medium/enig...,0.858824
423,0,69,True,2018-09-22 10:12:46.383000000,NaN,/training/hard/remainder-fantasy,0,NaN,NaN,hard,...,Remainder Fantasy,NaN,CODE,0,50.0,4.363636,11,https://www.codingame.com,https://www.codingame.com/training/hard/remain...,0.855072
422,0,210,True,2018-09-19 04:01:15.050000000,NaN,/training/medium/maximum-sub-sequence,0,NaN,NaN,medium,...,Maximum sub-sequence,NaN,CODE,0,50.0,4.394737,38,https://www.codingame.com,https://www.codingame.com/training/medium/maxi...,0.847619


# Write README.md

In [19]:
from tabulate import tabulate

In [20]:
df["url"] = df["url"].apply(lambda x:"[Link]({})".format(x))

In [21]:
def to_cat(x):
    pts = {
        "medium" : 2,
        "hard" : 3,
        "easy" : 1,
        "expert" : 4,
        "multi" : 7,
        "optim" : 6,
        "codegolf" : 5,
        "tutorial" : 0
    }
    return pts[x]

df["diff"] = df["level"].apply(to_cat)

In [22]:
def to_emojis(x):
    if x==100:
        return ":heavy_check_mark:"  
    elif x>0: 
        return ":white_check_mark:" 
    else: 
        return ""

df["Done"] = df["validatorScore"].apply(to_emojis)

In [23]:
df = df.sort_values(["diff", "validatorScore"], ascending=[True, False])

In [24]:
sub = df[["title", "level", "Done", "url"]]
sub = sub.set_index("title")
txt = tabulate(sub, tablefmt="pipe", headers="keys")

In [25]:
with open("README.md", "w") as f:
    f.write(txt)

# Download solutions

In [21]:
import json
import requests
from pprint import pprint

In [22]:
cookies = {"cgSession":""}

In [28]:
# data = ["20730215e9a4091099765b3747c71c253e7a89c3"]
# data = [df.loc[5, "testSessionHandle"]]
# r = requests.post('https://www.codingame.com/services/TestSession/startTestSession', cookies=cookies, data = str(data))
# r.text
# js = json.loads(r.text)
# pprint(js)

In [29]:
# code = js["currentQuestion"]["answer"]["code"]
# language = js["currentQuestion"]["answer"]["programmingLanguageId"]

In [30]:
# data = ["20730215e9a4091099765b3747c71c253e7a89c3", "Python3"]
# data = [df.loc[5, "testSessionHandle"], "Python3"]
# r = requests.post('https://www.codingame.com/services/TestSession/getPreviousCodeByLanguageId', cookies=cookies, data = str(data))
# r.text
# js = json.loads(r.text)
# pprint(js)

In [23]:
def extract_code(testSessionHandle, language="Python3"):
    data = [testSessionHandle, language]
    r = requests.post('https://www.codingame.com/services/TestSession/getPreviousCodeByLanguageId', cookies=cookies, data = str(data))
    code = json.loads(r.text)
    return code

In [24]:
import os
for row in df.iterrows():
    testSessionHandle = row[1]["testSessionHandle"]
    validatorScore = row[1]["validatorScore"]
    level = row[1]["level"]
    root, folder, title = row[1]["detailsPageUrl"][1:].split("/")
    title = title.replace(":", "")
    if validatorScore == 0:
        continue
    extension = {
        "Python3" : "py",
        "Javascript" : "js",
    }
    language = "Python3"
    filename = "{}/{}/{}.{}".format(root, folder, title, extension[language])
    if os.path.isfile(filename):
        continue
    if not os.path.exists("{}/{}".format(root, folder)):
        os.makedirs("{}/{}".format(root, folder))
    print("Extracting {} : {}".format(testSessionHandle, filename))
    code = extract_code(testSessionHandle, language=language)
    if code is None:
        continue
    with open(filename, "w") as f : 
        f.write(code)

Extracting 20745864f048f8f1593723ef1ef68f9ad4b9b837 : training/easy/a-childs-play.py
Extracting 20751565d5d17887f08106c5791d11270deb448e : training/medium/detective-pikaptcha-ep3.py
Extracting 208277035034093f884ed9c0a95bad472f49727e : training/hard/detective-pikaptcha-ep4.py
Extracting 21130542eae5c5bb21bb3f39537be75b08d15aa4 : training/easy/happy-numbers.py
Extracting 21130622b5e8d41096e26f95a6b3d8eab7fb93cd : training/easy/dolbears-law.py
